# Two Sample t-test
Chaitanya Anand <br>
(https://github.com/decisive-moment) <br>
Created on: Jan 04, 2020 <br>
Last Edit: March 03, 2020 <br>
Notebook to demonstrate two sample t-test using R. <br> Deliberately not used R's inbuilt t.test function in order to explore the inner workings of the test. <br><br>
This notebook assumes that the reader is familiar with the theory of hypothesis testing. If not then I would recommend reading up on the basics before going through the code. <br><br>
Some resources that I have found useful for learning hypothesis testing are: <br>(By no means an exhaustive list. One may pick up any resource on introductry statistics) <br>
- Design and Analysis of Experiments by Montgomery ISBN-13: 978-1118146927
- Coursera course on inferential statistics: https://www.coursera.org/learn/inferential-statistics/home/welcome

In [0]:
# Loading important libraries
library(dplyr)
library(ggplot2)

### Important Formulas

Degrees of freedom for the equal variance case<br>
$
\begin{align}
df = n_1 + n_2 - 2 \\
\end{align}
$<br>

Degrees of freedom for the unequal variance case<br>
$
\begin{align}
df = \frac{\left(\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}\right)^2}{\frac{\left(\frac{s_1^2}{n_1}\right)^2}{n_1-1} + \frac{\left(\frac{s_2^2}{n_2}\right)^2}{n_2-1}}\\
\end{align}
$<br>

Standard deviation for the equal variance case<br>
$
\begin{align}
s_{\bar{X_1}-\bar{X_2}}=\sqrt{\frac{(n_1-1)s_1^2-(n_2-1)s_2^2}{n_1+n_2-2}}\\
\end{align}
$<br>

Standard deviation for the unequal variance case<br>
$
\begin{align}
s_{\bar{X_1}-\bar{X_2}}=\sqrt{s_1^2+s_2^2}
\end{align}
$

Standard error for the equal variance case<br>
$
\begin{align}
SE_{\bar{X_1}-\bar{X_2}}=\sqrt{\left(\frac{(n_1-1)s_1^2-(n_2-1)s_2^2}{n_1+n_2-2}\right)\left(\frac{1}{n_1}+\frac{1}{n_2}\right)}\\
\end{align}
$<br>

Standard error for the unequal variance case<br>
$
\begin{align}
SE_{\bar{X_1}-\bar{X_2}}=\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}
\end{align}
$

### Data Generation Process

In [0]:
#### -------- Parameters of the experiment
# Number of trials from the first random process
n1 = 100
# Number of trials from the second random process
n2 = 100
# Chosen significance level
alpha = 0.05
# Assume equal variance of the two samples?
eqvar = FALSE
# Two tailed or one tailed test?
twotail = TRUE

#### -------- Population parameters
# Actual population mean of the first random process
mu1 = 11
# Actual population mean of the second random process
mu2 = 12
# Actual population standard deviation of the first random process
sigma1 = 3
# Actual population standard deviation of the econd random process
sigma2 = 2

In [0]:
#### -------- Let us say have collected some data from the two random processes
# The rnorm function in r helps us simulate exactly this
# The function will generate 'n' random values drawn from a
# normal distribution of the specified parameters (mu and sigma)

treatment1_data <- rnorm(n = n1, mean = mu1, sd = sigma1)
treatment2_data <- rnorm(n = n2, mean = mu2, sd = sigma2)

In [0]:
# We shall treat the population parameters as unknowm
# throughout the hypothesis testing process
# their only purpose was to obtain the simulated data
# in the previous step

# Deleting them from memory
rm(mu1, mu2, sigma1, sigma2)

### Stating the  Hypothesis
Let us say we have collected some data from two treatments (variations) of a random process. <br>
e.g. Measuring the height of plants where one group of seedlings is grown in clay soil (treatment 1) and another group is grown in loamy soil (treatment 2). We now want to detect if clay soil (treatment 1) and loamy soil (treatment 2) differ in their effect on plant growth.

The null hypothesis in this case would be:
\begin{align}
H_0: \mu_1 - \mu_2 = 0
\end{align}

The alternative hypothesis can be stated as (assuming we have no reason to believe one type of soil is better than the other):
\begin{align}
H_1: \mu_1 - \mu_2 \neq 0
\end{align}

### Calculating Sample Statistics

In [63]:
# Data collected from random process 1
glimpse(treatment1_data)

 num [1:100] 11.3 10.6 13.3 14.3 11.3 ...


In [64]:
# Data collected from random process 2
glimpse(treatment2_data)

 num [1:100] 11.88 11.9 12.94 8.33 11.17 ...


In [0]:
# Sample mean of data collected from process 1
x1_bar = mean(treatment1_data)
# Sample mean of data collected from process 2
x2_bar = mean(treatment2_data)

# Sample standard deviation of data collected from process 1
s1 = sd(treatment1_data)
# Sample standard deviation of data collected from process 2
s2 = sd(treatment2_data)

# Estimate of the population standard deviation
# Obtained using the sample standard deviations
# Since we are now treating the actual population parameters as unknown
# (please see the section on important formulas at the begining of this document)

var = ifelse(eqvar,

              # Equal standard deviation case
              # (i.e. assuming that the two process have the same standard deviation)
              (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))),

              # Unequal standard deviation case
              # (i.e. assuming that the two process have different standard deviation)
              (s1^2) + (s2^2)
)
sd = var^(1/2)


# Standard error
# (please see the section on important formulas at the begining of this document)
se = ifelse(eqvar,

              # Equal standard deviation case
              # (i.e. assuming that the two process have the same standard deviation)
              (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))*((1/n1) + (1/n2)))^(1/2),

              # Unequal standard deviation case
              # (i.e. assuming that the two process have different standard deviation)
              (((s1^2)/n1) + ((s2^2)/n2))^(1/2)
)


# Degrees of freedom
# (please see the section on important formulas at the begining of this document)
# Equal standard deviation case
df = ifelse(eqvar,

              # Equal standard deviation case
              # (i.e. assuming that the two process have the same standard deviation)
              n1 + n2 - 2,

              # Unequal standard deviation case
              # (i.e. assuming that the two process have different standard deviation)
              (((s1^2/n1) + (s2^2/n2))^2)/
                (((s1^2/n1)^2/(n1-1)) + (((s2^2/n2)^2)/(n2-1)))
)

# Observed effect (diffference between the two treatments i.e. random processes)
observed_dfference = x1_bar - x2_bar
# z value of the observed effect
# Since null hypothesis is that the difference in means is zero
# The sampling distribution is centered at mu = 0
z = (0 - observed_dfference)/se

In [66]:
x1_bar
x2_bar
x1_bar-x2_bar

[1] 10.64781

[1] 12.00728

[1] -1.359469

In [67]:
s1
s2

[1] 2.553093

[1] 1.867611

In [68]:
se
z

[1] 0.3163267

[1] 4.297674

### Step by Steb Procedure

H0: Treatment 1 and Treatment 2 are not diferent i.e. mu1 = mu2 or mu1 - mu2 = 0 <br>
H1: Treatment 1 and Treatment 2 are the same i.e. mu1 != mu2 or mu1 - mu2 != 0 <br>

In [76]:
p = pt(q = z, df = df, lower.tail = TRUE, log.p = FALSE)
p = ifelse(twotail,
            ifelse(p > 0.5, (1-p), p)*2,
            ifelse(p > 0.5, (1-p), p)
)
format(p, scientific = TRUE)
format(round(p, 5), scientific = FALSE)

[1] "2.812031e-05"

[1] "0.00003"

In [70]:
ifelse(p < alpha, "We reject the null hypothesis", "We fail to reject the null hypothesis")

[1] "We reject the null hypothesis"

### Alternative Approaoch - Confidence Interval

In [0]:
critical_value = qt(p = alpha/2, df = df, lower.tail = TRUE, log.p = FALSE)
lower_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                 observed_dfference - (critical_value*se),
                 observed_dfference + (critical_value*se))
upper_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                 observed_dfference + (critical_value*se),
                 observed_dfference - (critical_value*se))
ci = c(lower_limit, observed_dfference, upper_limit)

In [78]:
ci

[1] -1.9836226 -1.3594689 -0.7353151

In [79]:
ifelse(0 >= ci[1] & 0 <= ci[3],
       "The confidence interval contains zero",
       "The confidence interval (corresponding to chosen alpha) does not contain 0")

[1] "The confidence interval (corresponding to chosen alpha) does not contain 0"

### Verifying the results using R's inbuilt t.test function

In [80]:
t.test(treatment1_data,
       treatment2_data,
       mu = 0,
       paired = FALSE,
       var.equal = eqvar,
       alternative = ifelse(twotail, "two.sided", "one.sided"),
       conf.level = 1 - alpha)


	Welch Two Sample t-test

data:  treatment1_data and treatment2_data
t = -4.2977, df = 181.37, p-value = 2.812e-05
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.9836226 -0.7353151
sample estimates:
mean of x mean of y 
 10.64781  12.00728 


### Encapsulating the above process in a function (to enable iterative experimentation)

In [0]:
two_sample_t <- function(n1, n2, alpha, mu1, mu2, sigma1, sigma2, eqvar, twotail) {
    # Generating data
    treatment1_data <- rnorm(n = n1, mean = mu1, sd = sigma1)
    treatment2_data <- rnorm(n = n2, mean = mu2, sd = sigma2)
    

    # Sample mean of data collected from process 1
    x1_bar = mean(treatment1_data)
    # Sample mean of data collected from process 2
    x2_bar = mean(treatment2_data)

    # Sample standard deviation of data collected from process 1
    s1 = sd(treatment1_data)
    # Sample standard deviation of data collected from process 2
    s2 = sd(treatment2_data)

    # Estimate of the population standard deviation
    # Obtained using the sample standard deviations
    # Since we are now treating the actual population parameters as unknown
    # (please see the section on important formulas at the begining of this document)

    var = ifelse(eqvar,

                  # Equal standard deviation case
                  # (i.e. assuming that the two process have the same standard deviation)
                  (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))),

                  # Unequal standard deviation case
                  # (i.e. assuming that the two process have different standard deviation)
                  (s1^2) + (s2^2)
    )
    sd = var^(1/2)


    # Standard error
    # (please see the section on important formulas at the begining of this document)
    se = ifelse(eqvar,

                  # Equal standard deviation case
                  # (i.e. assuming that the two process have the same standard deviation)
                  (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))*((1/n1) + (1/n2)))^(1/2),

                  # Unequal standard deviation case
                  # (i.e. assuming that the two process have different standard deviation)
                  (((s1^2)/n1) + ((s2^2)/n2))^(1/2)
    )


    # Degrees of freedom
    # (please see the section on important formulas at the begining of this document)
    # Equal standard deviation case
    df = ifelse(eqvar,

                  # Equal standard deviation case
                  # (i.e. assuming that the two process have the same standard deviation)
                  n1 + n2 - 2,

                  # Unequal standard deviation case
                  # (i.e. assuming that the two process have different standard deviation)
                  (((s1^2/n1) + (s2^2/n2))^2)/
                    (((s1^2/n1)^2/(n1-1)) + (((s2^2/n2)^2)/(n2-1)))
    )

    # Observed effect (diffference between the two treatments i.e. random processes)
    observed_dfference = x1_bar - x2_bar
    # z value of the observed effect
    # Since null hypothesis is that the difference in means is zero
    # The sampling distribution is centered at mu = 0
    z = (0 - observed_dfference)/se












    # Calculating p-value
    p = pt(q = z, df = df, lower.tail = TRUE, log.p = FALSE)
    p = ifelse(twotail,
                ifelse(p > 0.5, (1-p), p)*2,
                ifelse(p > 0.5, (1-p), p)
    )
    # format(p, scientific = TRUE)
    # format(round(p, 5), scientific = FALSE)
    
    # Calculating the cnfidence interval
    critical_value = qt(p = alpha/2, df = df, lower.tail = TRUE, log.p = FALSE)
    lower_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                    observed_dfference - (critical_value*se),
                    observed_dfference + (critical_value*se))
    upper_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                    observed_dfference + (critical_value*se),
                    observed_dfference - (critical_value*se))
    ci = c(lower_limit, observed_dfference, upper_limit)
    
    # Results of the hypothesis test
    null_hypothesis_test_method <- ifelse(p <= alpha, "We reject the null hypothesis", "We fail to reject the null hypothesis")
    confidence_interval_method <- ifelse(ci[1] <= 0 & 0 <= ci[3],
       "The confidence interval contains zero",
       "The confidence interval (corresponding to chosen alpha) does not contain 0")
    
    ret <- list(ifelse(null_hypothesis_test_method == "We reject the null hypothesis" &
                  confidence_interval_method == "The confidence interval (corresponding to chosen alpha) does not contain 0", "Reject",
                  
                  ifelse(null_hypothesis_test_method == "We fail to reject the null hypothesis" &
                         confidence_interval_method == "The confidence interval contains zero", "Fail to reject",
                         "ERROR")),
                p = p,
                ci = ci,
                x1_bar = x1_bar,
                x2_bar = x2_bar,
                s1 = s1,
                s2 = s2,
                se = se)
    
    return(ret)
}

In [83]:
# Parameters of the experiment
exp1_n1 = 100
exp1_n2 = 100
exp1_alpha = 0.05
exp1_eqvar = FALSE
exp1_twotail = TRUE

# Population parameters

exp1_mu1 = 12
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3

two_sample_t(n1 = exp1_n1,
             n2 = exp1_n2,
             alpha = exp1_alpha,
             mu1 = exp1_mu1,
             mu2 = exp1_mu2,
             sigma1 = exp1_sigma1,
             sigma2 = exp1_sigma2,
             eqvar = exp1_eqvar,
             twotail = exp1_twotail)

[[1]]
[1] "Fail to reject"

$p
[1] 0.1252101

$ci
[1] -0.1876336  0.6683944  1.5244223

$x1_bar
[1] 11.81131

$x2_bar
[1] 11.14291

$s1
[1] 3.170339

$s2
[1] 2.964979

$se
[1] 0.4340755

In [85]:
# Iterate

# Parameters of the experiment
exp1_n1 = 100
exp1_n2 = 100
exp1_alpha = 0.05
exp1_eqvar = FALSE
exp1_twotail = TRUE

# Population parameters

exp1_mu1 = 12
exp1_mu2 = 12
exp1_sigma1 = 3
exp1_sigma2 = 3



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

[[1]]
[1] "Reject"

$p
[1] 0.03691539

$ci
[1] 0.05870906 0.95669473 1.85468039

$x1_bar
[1] 12.16177

$x2_bar
[1] 11.20507

$s1
[1] 2.909029

$s2
[1] 3.502085

$se
[1] 0.4552697

[[1]]
[1] "Reject"

$p
[1] 0.03260578

$ci
[1] 0.07589424 0.90719227 1.73849031

$x1_bar
[1] 12.70373

$x2_bar
[1] 11.79654

$s1
[1] 2.794583

$s2
[1] 3.155519

$se
[1] 0.4215091

[[1]]
[1] "Reject"

$p
[1] 0.007800936

$ci
[1] -1.9048316 -1.0987429 -0.2926542

$x1_bar
[1] 11.67175

$x2_bar
[1] 12.7705

$s1
[1] 2.906256

$s2
[1] 2.874441

$se
[1] 0.4087632

[[1]]
[1] "Reject"

$p
[1] 0.02821413

$ci
[1] 0.1065485 0.9873877 1.8682269

$x1_bar
[1] 12.47069

$x2_bar
[1] 11.4833

$s1
[1] 2.972353

$s2
[1] 3.333653

$se
[1] 0.4466332



results
Fail to reject         Reject 
            96              4 

In [86]:
# Iterate

# Parameters of the experiment
exp1_n1 = 1000000
exp1_n2 = 1000000
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 11
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

[[1]]
[1] "Reject"

$p
[1] 0.005041232

$ci
[1] 0.002603458 0.010927236 0.019251014

$x1_bar
[1] 11.00489

$x2_bar
[1] 10.99397

$s1
[1] 3.00011

$s2
[1] 3.005912

$se
[1] 0.004246901

[[1]]
[1] "Reject"

$p
[1] 0.01571685

$ci
[1] 0.0008124829 0.0091248264 0.0174371698

$x1_bar
[1] 11.00704

$x2_bar
[1] 10.99792

$s1
[1] 3.000365

$s2
[1] 2.997408

$se
[1] 0.004241067

[[1]]
[1] "Reject"

$p
[1] 0.004667369

$ci
[1] 0.002707692 0.011005263 0.019302835

$x1_bar
[1] 11.00925

$x2_bar
[1] 10.99824

$s1
[1] 2.99257

$s2
[1] 2.994545

$se
[1] 0.00423353

[[1]]
[1] "Reject"

$p
[1] 0.00723687

$ci
[1] -0.018685164 -0.010371846 -0.002058528

$x1_bar
[1] 10.99729

$x2_bar
[1] 11.00766

$s1
[1] 2.997972

$s2
[1] 3.000505

$se
[1] 0.004241564

[[1]]
[1] "Reject"

$p
[1] 0.01762342

$ci
[1] 0.0006176233 0.0089346401 0.0172516568

$x1_bar
[1] 11.0034

$x2_bar
[1] 10.99447

$s1
[1] 3.000972

$s2
[1] 3.000175

$se
[1] 0.004243451



results
         ERROR Fail to reject         Reject 
             6             89              5 

In [87]:
# Iterate

# Parameters of the experiment
exp1_n1 = 1000000
exp1_n2 = 1000000
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 11
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

results
         ERROR Fail to reject         Reject 
             4             94              2 

In [88]:
# Iterate

# Parameters of the experiment
exp1_n1 = 100
exp1_n2 = 100
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 12
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

results
         ERROR Fail to reject         Reject 
            14             25             61 

In [89]:
# Iterate

# Parameters of the experiment
exp1_n1 = 1000000
exp1_n2 = 1000000
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 11
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

[[1]]
[1] "Reject"

$p
[1] 0.02249163

$ci
[1] 0.0001903504 0.0085092823 0.0168282142

$x1_bar
[1] 11.00502

$x2_bar
[1] 10.99651

$s1
[1] 3.001731

$s2
[1] 3.000798

$se
[1] 0.004244428

[[1]]
[1] "Reject"

$p
[1] 0.02144019

$ci
[1] 0.0002752346 0.0085860563 0.0168968780

$x1_bar
[1] 11.00601

$x2_bar
[1] 10.99742

$s1
[1] 2.999806

$s2
[1] 2.99687

$se
[1] 0.004240291

[[1]]
[1] "Reject"

$p
[1] 0.01748825

$ci
[1] -0.0172590406 -0.0089448431 -0.0006306455

$x1_bar
[1] 10.99693

$x2_bar
[1] 11.00588

$s1
[1] 2.997627

$s2
[1] 3.001484

$se
[1] 0.004242013

[[1]]
[1] "Reject"

$p
[1] 0.02311591

$ci
[1] 0.0001413311 0.0084580946 0.0167748581

$x1_bar
[1] 11.00481

$x2_bar
[1] 10.99635

$s1
[1] 3.001224

$s2
[1] 2.99974

$se
[1] 0.004243322



results
         ERROR Fail to reject         Reject 
             3             93              4 